In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_date, year, lit,
    when, trim, regexp_replace
)
from pyspark.sql.types import (
    IntegerType, DoubleType, StringType
)

In [0]:
df_marketing = spark.sql("select * from nttdataeducacao.bronze.escolas_bronz")
display(df_marketing)

In [0]:
column_mapping = {
    "NU_ANO_CENSO": "ano",
    "NO_REGIAO": "regiao",
    "SG_UF": "uf",
    "CO_MUNICIPIO": "cod_munic",
    "NO_MUNICIPIO": "nome_munic",
    "CO_ENTIDADE": "cod_entid",
    "NO_ENTIDADE": "nome_entid",
    "NO_CATEGORIA": "localizacao",
    "NO_DEPENDENCIA": "dependencia",
    "ED_INF_CAT_0": "total_educacao_infantil",
    "CRE_CAT_0": "creche",
    "PRE_CAT_0": "pre_escola",
    "FUN_CAT_0": "total_ensino_fundamental",
    "FUN_AI_CAT_0": "anos_iniciais",
    "FUN_AF_CAT_0": "anos_finais",
    "FUN_01_CAT_0": "fundamental_1",
    "FUN_02_CAT_0": "fundamental_2",
    "FUN_03_CAT_0": "fundamental_3",
    "FUN_04_CAT_0": "fundamental_4",
    "FUN_05_CAT_0": "fundamental_5",
    "FUN_06_CAT_0": "fundamental_6",
    "FUN_07_CAT_0": "fundamental_7",
    "FUN_08_CAT_0": "fundamental_8",
    "FUN_09_CAT_0": "fundamental_9",
    "MED_CAT_0": "total_ensino_medio",
    "MED_01_CAT_0": "medio_1",
    "MED_02_CAT_0": "medio_2",
    "MED_03_CAT_0": "medio_3",
    "MED_04_CAT_0": "medio_4",
    "MED_NS_CAT_0": "nao_seriado",
}
 
for old_col, new_col in column_mapping.items():
    df_marketing = df_marketing.withColumnRenamed(old_col, new_col)
 

In [0]:
display(df_marketing)

In [0]:
from pyspark.sql.functions import year, col

df = df_marketing.withColumn("ano", year(col("ano")))
display(df)

In [0]:
# Vamos garantir que não haja texto de metadados na coluna Regiao.
df_sem_metadados = df_marketing.filter(
    # Filtra e mantém apenas as linhas onde a coluna Regiao NÃO CONTENHA a string "Fonte: "
    ~col("Regiao").contains("Fonte:")
)

# Se o texto "Fonte: Censo da Educação Básica 2024/INEP." estiver em outra coluna 
# que o usuário tentou converter para DATE, ele ainda pode persistir.
# Adicionar um filtro de segurança na coluna Ano para garantir que seja realmente numérico (INT > 0).
df_sem_metadados = df_sem_metadados.filter(
    col("ano") > 0
)

display(df_sem_metadados)

In [0]:
df_ready_to_write = df_sem_metadados.withColumn(
    "ano", 
    col("ano").cast(IntegerType()) # Ensure 'ano' is also an Integer
).withColumn(
    "total_educ_inf", # Column causing the error
    col("total_educ_inf").cast(IntegerType()) 
)
df_ready_to_write.write.mode("overwrite").saveAsTable("nttdataeducacao.silver.escolas_2024")